# VAE Mixed Likelihood GPLVM

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import time
import os

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from IPython import display
%matplotlib inline
import seaborn as sns

In [ ]:
sns.set()
sns.set_context("paper")

In [ ]:
import tfgp
from tfgp.util import data
from tfgp.model import VAEMLGPLVM
print(f"Succesfully imported package: {tfgp.__file__}")

## Generate data

In [ ]:
num_data = None
output_dim = 12
y, likelihood, labels = data.make_oilflow(num_data, output_dim)
if num_data is None:
    num_data = y.shape[0]

## Create model

In [ ]:
latent_dim = 10
num_inducing = 50
num_hidden = 50

In [ ]:
kernel = tfgp.kernel.ARDRBF(variance=0.5, gamma=0.5, xdim=latent_dim, name="kernel")
m = VAEMLGPLVM(y, latent_dim, kernel=kernel, likelihood=likelihood, num_inducing=num_inducing, num_hidden=num_hidden)
m.initialize()

## Build graph

In [ ]:
loss = tf.losses.get_total_loss()
learning_rate = 1e-3
with tf.name_scope("train"):
    trainable_vars = tf.trainable_variables()
    optimizer = tf.train.RMSPropOptimizer(learning_rate, name="RMSProp")
    train_all = optimizer.minimize(loss, 
                                   var_list=tf.trainable_variables(),
                                   global_step=tf.train.create_global_step(),
                                   name="train")
with tf.name_scope("summary"):
    m.create_summaries()
    tf.summary.scalar("total_loss", loss, family="Loss")
    for reg_loss in tf.losses.get_regularization_losses():
        tf.summary.scalar(f"{reg_loss.name}", reg_loss, family="Loss")
    merged_summary = tf.summary.merge_all()
init = tf.global_variables_initializer()

## Callback

In [ ]:
def plot(x: np.ndarray, *, z: np.ndarray = None, gammas: np.ndarray = None, loss) -> None:
    ax1.scatter(*x.T, c=labels, cmap="Paired")
    if z is not None:
        ax1.scatter(*z.T, c="k", marker="x")
    ax_x_min, ax_y_min = np.min(x, axis=0)
    ax_x_max, ax_y_max = np.max(x, axis=0)
    ax1.set_xlim(ax_x_min, ax_x_max)
    ax1.set_ylim(ax_y_min, ax_y_max)
    ax1.set_title(f"Step {i}")
    
    ax2.plot(*np.array(loss).T)
    ax2.set_title(f"Loss: {loss[-1][1]}")
    
    if gammas is not None:
        ax3.bar(range(len(gammas)), gammas, tick_label=(np.arange(len(gammas)) + 1))
    
    display.display(f)
    display.clear_output(wait=True)

## Setup optimisation

In [ ]:
root_dir = f"../.."
dataset = "oilflow"
start_time = f"{time.strftime('%Y%m%d%H%M%S')}"
log_dir = f"{root_dir}/log/{dataset}/{start_time}"
save_dir = f"{root_dir}/save/{dataset}/{start_time}"
output_dir = f"{root_dir}/output/{dataset}/{start_time}"
os.makedirs(save_dir)
os.makedirs(output_dir)

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
saver = tf.train.Saver()
#saver.restore(sess, f"{save_dir}/model.ckpt")

## Run optimisation

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
loss_list = []
n_iter = 100000
print_interval = 500
save_interval = 5000
batch_size = 100
all_data_indices = np.arange(num_data)
all_data_dict = {m.batch_indices: all_data_indices}
try:
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
    sess.run(init)
    for i in range(n_iter):
        batch_indices = np.random.choice(num_data, batch_size, replace=False)
        batch_dict = {m.batch_indices: batch_indices}
        sess.run(train_all, feed_dict=batch_dict)
        if i % print_interval == 0:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            train_loss, summary = sess.run([loss, merged_summary], feed_dict=all_data_dict, 
                                           options=run_options, run_metadata=run_metadata)
            summary_writer.add_run_metadata(run_metadata, f"step_{i}", global_step=i)
            summary_writer.add_summary(summary, i)
            gammas = sess.run(m.kernel._gamma)
            x_mean, _ = sess.run(m.encoder, feed_dict=all_data_dict)
            x_mean = x_mean[:, np.argsort(gammas)[-2:]]
            z = sess.run(m.z)
            loss_list.append([i, train_loss])
            plot(x_mean, gammas=gammas, loss=loss_list)
            ax1.cla()
            ax2.cla()
            ax3.cla()
        if i % save_interval == 0:
            saver.save(sess, f"{save_dir}/model.ckpt", global_step=i)
            np.savetxt(f"{output_dir}/x_mean_{i}.csv", x_mean)
            np.savetxt(f"{output_dir}/z_{i}.csv", z)
            np.savetxt(f"{output_dir}/labels.csv", labels)
            plot(x_mean, gammas=gammas, loss=loss_list)
            plt.savefig(f"{output_dir}/fig_{i}.eps")
            ax1.cla()
            ax2.cla()
            ax3.cla()
except KeyboardInterrupt:
    pass
finally:
    gammas = sess.run(m.kernel._gamma)
    x_mean, _ = sess.run(m.encoder, feed_dict=all_data_dict)
    x_mean = x_mean[:, np.argsort(gammas)[-2:]]
    z = sess.run(m.z)
    loss_list.append([i, sess.run(loss, feed_dict=all_data_dict)])
    plot(x_mean, gammas=gammas, loss=loss_list)


## PCA

In [ ]:
x_pca = tfgp.util.pca_reduce(y, 2)
plt.scatter(*x_pca.T, c=labels, cmap="Paired")

## Compute 1NN error

In [ ]:
k = 2
err_mlgplvm = tfgp.util.knn_error(x_mean, labels, k)
err_pca = tfgp.util.knn_error(x_pca, labels, k)
print(f"Missclasifications with MLGPLVM: {err_mlgplvm}")
print(f"Missclasifications with PCA: {err_pca}")

# Save figures

In [ ]:
#plt.scatter(*x_mean.T, c=labels, cmap="Paired")
plt.savefig(f"{output_dir}/{dataset}_mlgplvm.eps", format="eps", dpi=1000)

In [ ]:
plt.bar(range(len(gammas)), gammas, tick_label=(np.arange(len(gammas)) + 1))
plt.savefig(f"{output_dir}/{dataset}_gamma.eps", format="eps", dpi=1000)

# REGENERATE IMAGES

In [ ]:
time_stamp = "20180926140944"

In [ ]:
saver.restore(sess, f"{root_dir}/save/{dataset}/{time_stamp}/model.ckpt-15000")

In [ ]:
labels = np.loadtxt(f"{root_dir}/output/{dataset}/{time_stamp}/labels.csv")
gammas = m.kernel._gamma.eval()
x_mean = m.qx_mean.eval()
x_mean = x_mean[:, np.argsort(gammas)[-2:]]

In [ ]:
plt.figure(figsize=(2,2))
#plt.tick_params(
#    axis='both',          # changes apply to the x-axis
#    which='both',      # both major and minor ticks are affected
    #bottom=False,      # ticks along the bottom edge are off
    #top=False,         # ticks along the top edge are off
    #left=False,
    #right=False,
#    labelbottom=False,
#    labelleft=False,)
plt.scatter(*x_mean.T, c=labels, cmap="Paired", s=5)
plt.locator_params(nbins=2)
#plt.xlabel("Dimension 1")
#plt.ylabel("Dimension 9")
#plt.savefig(f"{root_dir}/{dataset}_mlgplvm.eps", format="eps", dpi=1000, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(2,2))
plt.bar(range(len(gammas)), gammas, tick_label=(np.arange(len(gammas)) + 1))
#plt.xlabel("Dimension")
plt.savefig(f"{root_dir}/{dataset}_gamma.eps", format="eps", dpi=1000, bbox_inches='tight')